# EXIF Wrapped

Analyzer for photo metadata.

Summarize your photo-taking habits along important metrics: `Focal Length`, `Aperature`, `ISO`, `Shutter Speed`.

## 1. Quickstart: Define local variables

1. Set `camera_model` to the name of your camera, including any spaces necessary (not case sensitive). 
2. Set the `directory` to the base folder of where your images are located.
    - Hint: Change directory `cd` to the deepest/innermost (child-most) folder that still contains *all* of your photos. Use `pwd` to display the path to that directory, and set it to `directory`.
3. (*optional*) Set `output_dir` to save your data to .csv format and plots as images. 

In [ ]:
camera_model = 'Nikon Z 5'                                          # Replace with camera name
directory = '/Users/derekzheng/Pictures/Lightroom Saved Photos'     # Replace with path to images
output_dir = '/Users/derekzheng/Documents/Code/dz-img-analysis'     # (optional) Specify output file destination

## 2. Run!

After completion of the above, hit `Run All` and you should see your results at the bottom!

In [ ]:
# Import basics
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from PIL.ExifTags import TAGS
from fractions import Fraction

EXIF_DIMS = ['Focal Length', 'Aperture', 'ISO', 'Shutter Speed']

In [ ]:
def get_exif_data(image_path):
    image = Image.open(image_path)
    exif_data = {}
    info = image._getexif()
    if info:
        for tag_id, value in info.items():
            tag = TAGS.get(tag_id, tag_id)
            exif_data[tag] = value
    return exif_data

In [ ]:
other_cameras_found = set()

def analyze_photos(directory, camera_model=None):
    data = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.tiff')):
                file_path = os.path.join(root, file)
                try:
                    with Image.open(file_path) as image:
                        exif_data = {}
                        info = image._getexif()
                        if info:
                            for tag_id, value in info.items():
                                tag = TAGS.get(tag_id, tag_id)
                                exif_data[tag] = value

                        if camera_model and exif_data.get('Model').lower() != camera_model.lower():
                            # print(f"Skipping, file {file}, is from {exif_data.get('Model')}")
                            other_cameras_found.add(exif_data.get('Model'))
                            continue

                        focal_length = exif_data.get('FocalLength', (0, 1))
                        focal_length = focal_length[0] / focal_length[1] if isinstance(
                            focal_length, tuple) else focal_length

                        aperture = exif_data.get('FNumber', (0, 1))
                        aperture = aperture[0] / aperture[1] if isinstance(
                            aperture, tuple) else aperture

                        iso = exif_data.get('ISOSpeedRatings')

                        shutter_speed = exif_data.get('ExposureTime', (0, 1))
                        shutter_speed = shutter_speed[0] / shutter_speed[1] if isinstance(
                            shutter_speed, tuple) else shutter_speed

                        data.append({
                            'Focal Length': focal_length,
                            'Aperture': aperture,
                            'ISO': iso,
                            'Shutter Speed': shutter_speed
                        })
                except Exception as e:
                    print(f"Error processing {file}: {str(e)}")

    return pd.DataFrame(data)

In [ ]:
def plot(directory, camera_model=None, output_dir=None):
    print(f"Analyzing photos in: {directory}")
    if camera_model:
        print(f"Filtering for camera model: {camera_model}")
    
    df = df_new
    
    if df.empty:
        print("No photos found matching the criteria.")
        return
    
    print(f"Processed {len(df)} photos.")
    print(f"Other cameras found: {other_cameras_found}")
    
    # Calculate statistics
    stats = df.describe(include='all')
    print("\nStatistics:")
    print(stats)

        # Plot histograms
    fig, axs = plt.subplots(2, 2, figsize=(15, 15))
    df['Focal Length'].hist(ax=axs[0, 0], bins=20)
    axs[0, 0].set_title('Focal Length Distribution')
    axs[0, 0].set_xlabel('Focal Length (mm)')
    
    df['Aperture'].hist(ax=axs[0, 1], bins=20)
    axs[0, 1].set_title('Aperture Distribution')
    axs[0, 1].set_xlabel('Aperture (f-number)')
    
    df['ISO'].hist(ax=axs[1, 0], bins=20)
    axs[1, 0].set_title('ISO Distribution')
    axs[1, 0].set_xlabel('ISO')
    
    df['Shutter Speed'].hist(ax=axs[1, 1], bins=20)
    axs[1, 1].set_title('Shutter Speed Distribution')
    axs[1, 1].set_xlabel('Shutter Speed (seconds)')
    
    plt.tight_layout()
    
    if output_dir:
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        plt.savefig(os.path.join(output_dir, 'photo_stats.png'))
        df.to_csv(os.path.join(output_dir, 'photo_data.csv'), index=False)
        stats.to_csv(os.path.join(output_dir, 'photo_stats.csv'))
    else:
        plt.show()

### Optional - Analysis of errors

Sometimes there are some photos taken on different devices, with different formats, or no EXIF data at all. You'll see any errors in the analysis phase below:

In [ ]:
df_cache = analyze_photos(directory, camera_model)

## 3. Results
- View high level statistics such as averages, mode, median for all standard dimensions.
- View histogram plot for all standard dimensions.
- View histogra plots for a specific dimension.

In [ ]:
def rational_to_float(value):
    if isinstance(value, tuple) and len(value) == 2:
        return value[0] / value[1]
    return float(value)

df_new = df_cache.copy()

# Convert columns to float
for column in EXIF_DIMS:
    df_new[column] = df_new[column].apply(rational_to_float)

# Now calculate statistics
columns = df_new.columns

for column in columns:
    print(f"\nStatistics for {column}:")
    
    # Mean
    mean = df_new[column].mean()
    print(f"Mean: {mean:.2f}")
    
    # Mode
    mode = df_new[column].mode().values
    if len(mode) == 1:
        print(f"Mode: {mode[0]:.2f}")
    else:
        print(f"Mode: {', '.join([f'{m:.2f}' for m in mode])}")
    
    # Median
    median = df_new[column].median()
    print(f"Median: {median:.2f}")

In [ ]:
# Histogram plots for all dimensions
plot(directory, camera_model=camera_model, output_dir=None)

### Optional: Dive deeper into a specific dimension

Here, we have an example in which we want to see a histogram for `Aperature` but within the specific range of `<4.0`. We can set `upper_trim_value` to `4.0`.

In [ ]:
# Edit these to your liking!
trim_dimension = 'Aperture'               # 'Focal Length', 'Aperture', 'ISO', or 'Shutter Speed'
upper_trim_value = 4.0                    # Value dependent on trim_dimension (i.e. under 80mm, under f/4, ISO 16000, or Shutter 1/4)

assert(trim_dimension in EXIF_DIMS)     

In [ ]:
df_trim = df_new[trim_dimension].copy()
df_trim.head()
df_trim = df_trim[df_trim < upper_trim_value]

In [ ]:
plt.figure(figsize=(5,5))
df_trim.hist(bins=20)
plt.title(f'{trim_dimension} Distribution, under {upper_trim_value}')
plt.xlabel(f'{trim_dimension}')

plt.show()